# Heston Pricing Example – SPY Surface

This notebook uses your `HestonModel`, `calibrate`, and `Pricer` classes with a SPY-style volatility surface from `SPY_Calibration_Template.xlsx`.


In [ ]:
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from heston_model import HestonModel
from heston_calibration import calibrate, bs_implied_vol, get_vol_slice
from heston_pricer import Pricer

warnings.filterwarnings("ignore")


### 1. Calibration on SPY Vol Surface

We load the SPY-style calibration template, build the risk-free and dividend yield curves, and calibrate the Heston model to the implied volatility surface for a 1-year maturity.

In [ ]:
# Initial Heston parameters (starting guess)
init_params = {
    "kappa": 1.0,
    "theta": 0.04,
    "xi": 0.4,
    "rho": -0.7,
    "v0": 0.05,
}
model = HestonModel(init_params)

# Input data: SPY-style calibration template
file = r"SPY_Calibration_Template.xlsx"
market_data = pd.read_excel(file, sheet_name="Market_Data")
surf = pd.read_excel(file, sheet_name="Vol_Matrix", index_col=0)

# Choose calibration maturity (in years)
T = 1.0  # 1-year maturity

# Risk-free rate curve
rate_curve = (
    market_data[["Year_Frac", "Risk_Free_Rate"]]
    .drop_duplicates()
    .sort_values("Year_Frac")
)
times_rf = rate_curve["Year_Frac"].values
rates = rate_curve["Risk_Free_Rate"].values
r = np.interp(T, times_rf, rates)

# Dividend yield curve
div_curve = (
    market_data[["Year_Frac", "Div_Yield"]]
    .drop_duplicates()
    .sort_values("Year_Frac")
)
times_div = div_curve["Year_Frac"].values
divs = div_curve["Div_Yield"].values
q = np.interp(T, times_div, divs)

# Spot price
S0 = float(market_data["S0"].iloc[0])

# Calibration (prints calibration details from calibrate())
res = calibrate(model, surf, S0, r, T, q)
res


### 1A. Volatility Surface Heatmap

Heatmap of the implied volatility surface used for calibration.

In [ ]:
plt.figure(figsize=(12, 6))
plt.imshow(surf.values, aspect="auto", cmap="viridis", origin="lower")
plt.colorbar(label="Implied Volatility")
plt.xticks(ticks=range(len(surf.columns)), labels=surf.columns.astype(str), rotation=45)
plt.yticks(ticks=range(len(surf.index)), labels=surf.index.astype(str))
plt.xlabel("Moneyness (%)")
plt.ylabel("Maturity (Years)")
plt.title("SPY Implied Volatility Surface Heatmap")
plt.tight_layout()
plt.show()


### 1B. 3D Implied Volatility Surface

3D surface plot of implied volatility as a function of moneyness and maturity.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

X = surf.columns.astype(float)
Y = surf.index.astype(float)
X_grid, Y_grid = np.meshgrid(X, Y)
Z = surf.values

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection="3d")
surf_plot = ax.plot_surface(X_grid, Y_grid, Z, cmap="viridis", edgecolor="none")

ax.set_title("SPY 3D Implied Volatility Surface")
ax.set_xlabel("Moneyness (%)")
ax.set_ylabel("Maturity (Years)")
ax.set_zlabel("Implied Volatility")

fig.colorbar(surf_plot, shrink=0.5, aspect=10)
plt.tight_layout()
plt.show()


### 2. Volatility Smiles – Heston vs Market

For a set of maturities, we compare the market implied volatility smile with the implied vols implied by Heston prices (via Black–Scholes inversion).

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

# Maturities to visualize
T_values = [0.5, 1.0, 1.5, 2.0]

for i, T_i in enumerate(T_values):
    moneyness, market_vols = get_vol_slice(surf, T_i)
    K_vals = moneyness * S0 / 100.0

    # Heston call prices at these strikes
    heston_prices = model.heston_call(T_i, S0, r, q, K_vals)

    # Convert Heston prices to implied vols via Black–Scholes
    heston_vols = [
        bs_implied_vol(S0, K, T_i, r, q, C)
        for K, C in zip(K_vals, heston_prices)
    ]

    ax = axes[i]
    ax.plot(moneyness, heston_vols, "o-", label="Heston Implied Vols")
    ax.plot(moneyness, market_vols, "x-", label="Market Implied Vols")
    ax.set_xlabel("Moneyness (%)")
    ax.set_ylabel("Implied Volatility")
    ax.set_title(f"Volatility Smile T = {T_i}")
    ax.legend()
    ax.grid(True)

fig.suptitle(
    f"Heston vs Market Volatility Smiles (Calibration T = {T})",
    fontweight="bold",
)
plt.tight_layout()
plt.show()


### 2A. Enhanced Smile Comparison (Single Maturity)

Detailed smile comparison at the calibration maturity with shaded error region between model and market implied volatilities.

In [ ]:
moneyness_T, market_vols_T = get_vol_slice(surf, T)
K_vals_T = moneyness_T * S0 / 100.0
heston_prices_T = model.heston_call(T, S0, r, q, K_vals_T)
heston_vols_T = [
    bs_implied_vol(S0, K, T, r, q, C)
    for K, C in zip(K_vals_T, heston_prices_T)
]

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(moneyness_T, market_vols_T, "o-", label="Market Smile", linewidth=2)
ax.plot(moneyness_T, heston_vols_T, "s--", label="Heston Smile", linewidth=2)

ax.fill_between(
    moneyness_T,
    market_vols_T,
    heston_vols_T,
    color="gray",
    alpha=0.2,
    label="Model Error Region",
)

ax.set_title(f"Volatility Smile Comparison (T = {T} Years)")
ax.set_xlabel("Moneyness (%)")
ax.set_ylabel("Implied Volatility")
ax.grid(True)
ax.legend()
plt.tight_layout()
plt.show()


### 3. Volatility Process Simulation

Simulate sample paths of the variance process under the calibrated Heston model and plot the volatility paths.

In [ ]:
S_paths, v_paths = model.simulate(S0, T, r, q, npaths=1000)

plt.figure(figsize=(8, 5))
plt.plot(np.sqrt(v_paths))  # plot volatility (sqrt of variance)
plt.xlabel("Time step")
plt.ylabel("Volatility")
plt.title("Simulated Volatility Paths (Heston)")
plt.grid(True)
plt.tight_layout()
plt.show()


### 3A. Volatility Fan Chart

Show the distribution of volatility over time using percentile bands (10%–90%).

In [ ]:
num_paths_to_plot = 300
subset_vol = np.sqrt(v_paths[:, :num_paths_to_plot])

p10 = np.percentile(subset_vol, 10, axis=1)
p50 = np.percentile(subset_vol, 50, axis=1)
p90 = np.percentile(subset_vol, 90, axis=1)

plt.figure(figsize=(10, 5))
plt.plot(p50, label="Median Volatility", color="black")
plt.fill_between(
    range(len(p10)),
    p10,
    p90,
    color="blue",
    alpha=0.3,
    label="10–90% Range",
)

plt.title("Heston Volatility Distribution Over Time")
plt.xlabel("Time Step")
plt.ylabel("Volatility")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


### 3B. Distribution of Terminal Prices

Histogram of simulated terminal prices $S_T$ to show the Heston distribution of SPY at maturity.

In [ ]:
plt.figure(figsize=(8, 5))
plt.hist(S_paths[-1, :], bins=60, density=True, alpha=0.7)
plt.title("Distribution of SPY Terminal Prices (Heston Simulation)")
plt.xlabel("S(T)")
plt.ylabel("Density")
plt.grid(True)
plt.tight_layout()
plt.show()


### 4. Call Prices Check – Monte Carlo vs Heston Formula vs Carr–Madan FFT

Compare call prices computed via Monte Carlo simulation, the semi-analytical Heston formula, and the Carr–Madan FFT method across a grid of strikes.

In [ ]:
# Strike grid
K_grid = np.linspace(0.5 * S0, 2.0 * S0, 20)

mc_prices = model.monte_carlo_call(T, S0, r, q, K_grid)
heston_prices_grid = model.heston_call(T, S0, r, q, K_grid)
fft_prices = model.carr_madan_call(T, S0, r, q, K_grid)

max_rel_err = np.max(np.abs(fft_prices / heston_prices_grid - 1.0))
print(
    f"Max relative error between Carr–Madan FFT and Heston semi-analytic: "
    f"{max_rel_err:.4e}"
)

plt.figure(figsize=(8, 5))
plt.plot(K_grid, mc_prices, "o-", label="Monte Carlo")
plt.plot(K_grid, heston_prices_grid, "s-", label="Heston (P1/P2)")
plt.plot(K_grid, fft_prices, "^-", label="Carr–Madan FFT")
plt.xlabel("Strike K")
plt.ylabel("Call Price")
plt.title("Call Prices under Heston Model (SPY Surface)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


### 5. Pricing Examples with `Pricer`

Use the `Pricer` class to price European, digital, and barrier options under the calibrated Heston model. The MTM values here are placeholders you can replace with real SPY quotes if you have them.

In [ ]:
K = 200.0      # strike
B = 150.0      # barrier level
n = 1000       # notional multiplier
seed = 2025
paths = 10**5
steps = 1000

pricer = Pricer(model)

# --- European Call ---
mtm = 21_522.81  # example MTM
call_price = pricer.european(T, S0, r, q, K, type="Call") * n
print(
    f"European Call Price: {call_price:.2f}; "
    f"Relative error: {(call_price / mtm - 1):.3%}"
)

# --- European Put ---
mtm = 26_149.40
put_price = pricer.european(T, S0, r, q, K, type="Put") * n
print(
    f"European Put Price: {put_price:.2f}; "
    f"Relative error: {(put_price / mtm - 1):.3%}"
)

# --- Digital Call ---
mtm = 395.72
digital_call_price = (
    pricer.digital(T, S0, r, q, K, type="Call", npaths=paths, seed=seed) * n
)
print(
    f"Digital Call Price: {digital_call_price:.2f}; "
    f"Relative error: {(digital_call_price / mtm - 1):.3%}"
)

# --- Digital Put ---
mtm = 571.86
digital_put_price = (
    pricer.digital(T, S0, r, q, K, type="Put", npaths=paths, seed=seed) * n
)
print(
    f"Digital Put Price: {digital_put_price:.2f}; "
    f"Relative error: {(digital_put_price / mtm - 1):.3%}"
)

# --- Barrier Up-and-In Call ---
mtm = 21_502.32
barrier_type = "UpAndIn"
barrier_price = (
    pricer.barrier(
        T,
        S0,
        r,
        q,
        K,
        B,
        barrier_type=barrier_type,
        option_type="Call",
        npaths=paths,
        nsteps=steps,
        seed=seed,
    )
    * n
)
print(
    f"Barrier {barrier_type} Price: {barrier_price:.2f}; "
    f"Relative error: {(barrier_price / mtm - 1):.3%}"
)

# --- Barrier Up-and-Out Call ---
mtm = 0.00  # placeholder
barrier_type = "UpAndOut"
barrier_price = (
    pricer.barrier(
        T,
        S0,
        r,
        q,
        K,
        B,
        barrier_type=barrier_type,
        option_type="Call",
        npaths=paths,
        nsteps=steps,
        seed=seed,
    )
    * n
)
rel_err_up_out = float("nan") if mtm == 0 else (barrier_price / mtm - 1)
print(
    f"Barrier {barrier_type} Price: {barrier_price:.2f}; "
    f"Relative error: {rel_err_up_out:.3%}"
)

# --- Barrier Down-and-Out Call ---
mtm = 20_164.98
barrier_type = "DownAndOut"
barrier_price = (
    pricer.barrier(
        T,
        S0,
        r,
        q,
        K,
        B,
        barrier_type=barrier_type,
        option_type="Call",
        npaths=paths,
        nsteps=steps,
        seed=seed,
    )
    * n
)
print(
    f"Barrier {barrier_type} Price: {barrier_price:.2f}; "
    f"Relative error: {(barrier_price / mtm - 1):.3%}"
)

# --- Barrier Down-and-In Call ---
mtm = 1_362.50
barrier_type = "DownAndIn"
barrier_price = (
    pricer.barrier(
        T,
        S0,
        r,
        q,
        K,
        B,
        barrier_type=barrier_type,
        option_type="Call",
        npaths=paths,
        nsteps=steps,
        seed=seed,
    )
    * n
)
print(
    f"Barrier {barrier_type} Price: {barrier_price:.2f}; "
    f"Relative error: {(barrier_price / mtm - 1):.3%}"
)


## Precomputed Heston Figures

The following figures are precomputed and stored as PNG files in the `figures/` directory, so they will render on GitHub without executing the notebook.

### Implied Volatility Surface

![Implied Volatility Surface](figures/heston_iv_surface.png)

---

### 1Y Smile: Market vs Heston Model

![1Y Smile: Market vs Heston Model](figures/heston_smile_fit.png)

---

### Synthetic Risk-Neutral Density

![Synthetic Risk-Neutral Density](figures/heston_density.png)


In [ ]:
from IPython.display import Image, display

print("Implied Volatility Surface (precomputed PNG)")
display(Image(filename="figures/heston_iv_surface.png"))

print("\n1Y Smile: Market vs Heston Model (precomputed PNG)")
display(Image(filename="figures/heston_smile_fit.png"))

print("\nSynthetic Risk-Neutral Density (precomputed PNG)")
display(Image(filename="figures/heston_density.png"))
